In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
directory_path = "/home/kwat/garden/data/cancer_cell_line/ctrp/"

kraft.path(directory_path)

In [ ]:
compound_table = pd.read_csv(
    "{}/v2/v20.meta.per_compound.txt".format(directory_path), sep="\t", index_col=1
)

kraft.error_axis(compound_table)

compound_table.index.name = "Compound"

compound_table

In [ ]:
gene_x_compound = kraft.binarize(compound_table["gene_symbol_of_protein_target"])

gene_x_compound.index.name = "Gene"

gene_x_compound.to_csv("{}gene_x_compound_x_01.tsv".format(directory_path), sep="\t")

gene_x_compound

In [ ]:
activity_x_compound = kraft.binarize(compound_table["target_or_activity_of_compound"])

activity_x_compound.index.name = "Activity"

activity_x_compound.to_csv(
    "{}/activity_x_compound_x_01.tsv".format(directory_path), sep="\t"
)

activity_x_compound

In [ ]:
cell_line_table = pd.read_csv(
    "{}v2/v20.meta.per_cell_line.txt".format(directory_path), sep="\t", index_col=1
)

kraft.error_axis(cell_line_table)

cell_line_table.index.name = "Cell Line"

cell_line_table

In [ ]:
mapping = pd.read_csv("{}new-abs-auc-with-qc.txt".format(directory_path), sep="\t")

mapping

In [ ]:
compound_x_cell_line = np.full(
    (compound_table.shape[0], cell_line_table.shape[0]), np.nan
)

compound_id_i = kraft.map_objects_to_ints(compound_table["master_cpd_id"])[0]

cell_line_id_i = kraft.map_objects_to_ints(cell_line_table["master_ccl_id"])[0]

for compound_id, cell_line_id, score in mapping[
    ["master_cpd_id", "master_ccl_id", "area_under_curve"]
].to_numpy():

    compound_x_cell_line[
        compound_id_i[int(compound_id)], cell_line_id_i[int(cell_line_id)]
    ] = score

compound_x_cell_line = pd.DataFrame(
    compound_x_cell_line, index=compound_table.index, columns=cell_line_table.index,
)

compound_x_cell_line = compound_x_cell_line.dropna(axis=1, how="all")

compound_x_cell_line.to_csv(
    "{}ctrp_2.compound_x_cell_line.tsv".format(directory_path), sep="\t"
)

compound_x_cell_line